The main solving big ridge regression problems

In [ ]:
workspace()

In [ ]:
using Plots

In [313]:
using StatsBase
using JLD  
include("../src/BigRidge.jl")

plot_outputs_Plots (generic function with 1 method)

In [314]:
#Basic parameters
## Basic parameters
maxiter=2000;
max_time = 200;
max_epocs = 100;
printiters = true;
exacterror =true; 
tol = 10.0^(-5.0);
restol = 10.0^(-5.0);
skip_error_calculation = 25;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))

In [317]:
dataset = "gisette_scale"   # news20.binary  rcv1_train.binary mnist.scale  a9a  gisette_scale  rcv1_train.binary
prob = defineProb(dataset);

../data/gisette_scale.jld
ErrorException("File ../data/gisette_scale-xsol.jld cannot be found")


In [318]:
norm(prob.A*prob.xsol - prob.b)
showDetails(dataset)

LoadError: KeyError: key "gisette_scale" not found

In [319]:
options = MyOptions(0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror,[]) 
OUTPUTS = [];

In [320]:
options.AUX = [0 5]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = 100; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

rademacher-10-2
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 42.30  |  12.36  | 0.1502 
 50  | 37.37  |  9.59  | 0.8480 
 75  | 33.85  |  8.56  | 0.9338 
100  | 31.38  |  7.81  | 1.0168 
125  | 28.83  |  6.52  | 1.1134 
150  | 27.04  |  6.00  | 1.1821 
175  | 25.72  |  5.71  | 1.2603 
200  | 23.63  |  5.87  | 1.3353 
225  | 21.11  |  4.75  | 1.4150 
250  | 20.92  |  4.58  | 1.4986 
275  | 19.75  |  4.52  | 1.5732 
300  | 18.38  |  4.05  | 1.6471 
325  | 17.70  |  3.95  | 1.7296 
350  | 16.54  |  3.77  | 1.8070 
375  | 16.20  |  3.51  | 1.8871 
400  | 15.14  |  3.78  | 1.9652 
425  | 14.89  |  3.30  | 2.0313 
450  | 14.50  |  3.26  | 2.1003 
475  | 14.05  |  2.97  | 2.1757 
500  | 13.24  |  2.90  | 2.2627 
525  | 12.86  |  2.86  | 2.3338 
550  | 12.51  |  2.80  | 2.4032 
575  | 12.10  |  2.65  | 2.4777 
600  | 11.88  |  2.51  | 2.5526 
625  | 11.49  |  2.54  | 2.6364 
650  | 11.21  |  2.39  | 2.7112 
675  | 10.92  |  2.32  | 2.7858 
700  | 10.52  | 

In [276]:
# Testing Hadamard Sketch
# method_name = "Hadamard";# ROS
# options.sketchsize =10; # Int(floor(log2(prob.n)));
# options.skip_error_calculation =25;
# @time  output= solveRidge(prob, method_name, options);
# OUTPUTS = [OUTPUTS ; output];

In [277]:
# Testing countmin 
# options.sketchsize =10;#floor((prob.n)^(1/2));
# options.skip_error_calculation =25;
# method_name = "countmin";# gradient descent
# @time output= solveRidge(prob, method_name, options);
# OUTPUTS = [OUTPUTS ; output];

In [321]:
# Testing CG 
method_name = "CG";# gradient descent
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CG
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 3.89  |  1.53  | 0.6564 
 50  | 1.76  |  0.67  | 1.2130 
 75  | 1.10  |  0.49  | 1.7700 
100  | 0.77  |  1.21  | 2.3274 
125  | 0.57  |  0.25  | 2.8970 
150  | 0.30  |  0.38  | 3.4821 
175  | 0.92  |  0.18  | 4.0527 
200  | 0.48  |  0.20  | 4.5608 
225  | 0.23  |  0.13  | 5.0683 
250  | 0.19  |  0.82  | 5.5513 
275  | 0.16  |  0.22  | 6.0433 
300  | 0.13  |  0.16  | 6.5322 
325  | 0.11  |  0.22  | 7.0176 
350  | 0.09  |  0.16  | 7.4789 
375  | 0.02  |  0.10  | 7.9893 
400  | 0.07  |  0.11  | 8.4833 
425  | 0.05  |  0.29  | 8.9930 
450  | 0.04  |  0.07  | 9.5145 
475  | 0.04  |  0.07  | 10.0201 
500  | 0.03  |  0.04  | 10.5416 
525  | 0.03  |  0.04  | 11.0502 
550  | 0.02  |  0.03  | 11.5566 
575  | 0.02  |  0.08  | 12.0656 
600  | 0.02  |  0.03  | 12.5730 
625  | 0.09  |  0.03  | 13.2212 
650  | 0.05  |  0.03  | 13.7721 
675  | 0.06  |  0.03  | 14.2876 
700  | 0.06  |  0.03  | 14.7978 
725  | 0.01  | 

In [ ]:
# Testing CD 
method_name = "CD";# gradient descent
options.sketchsize = 200;#floor((prob.n)^(1/2));
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CD-200
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 9.17  |  2.93  | 0.5781 
 50  | 5.60  |  1.26  | 0.9956 
 75  | 4.04  |  0.91  | 1.3728 
100  | 3.14  |  0.69  | 1.7714 
125  | 2.98  |  0.58  | 2.2041 
150  | 2.01  |  0.50  | 2.5667 
175  | 1.97  |  0.44  | 2.9538 
200  | 1.73  |  0.39  | 3.3187 
225  | 1.56  |  0.35  | 3.7083 
250  | 1.42  |  0.31  | 4.0870 
275  | 1.43  |  0.29  | 4.5077 
300  | 1.30  |  0.25  | 4.8632 
325

In [293]:
OUTPUTS =[];
include("../src/call_all_methods.jl")

rademacher-10-10-sgn
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 22.94  |  10.80  | 0.0881 
 50  | 15.23  |  7.68  | 0.1183 
 75  | 11.62  |  7.23  | 0.1505 
100  | 9.47  |  4.28  | 0.1775 
125  | 7.86  |  3.68  | 0.2122 
150  | 6.90  |  3.49  | 0.2398 
175  | 6.52  |  2.95  | 0.2746 
200  | 5.38  |  2.47  | 0.3025 
225  | 4.88  |  2.34  | 0.3327 
250  | 4.47  |  2.43  | 0.3597 
275  | 4.73  |  2.09  | 0.3896 
300  | 3.74  |  1.73  | 0.4171 
325  | 3.48  |  1.89  | 0.4473 
350  | 3.26  |  1.47  | 0.4743 
375  | 3.93  |  1.84  | 0.5042 
400  | 2.87  |  1.85  | 0.5317 
425  | 2.72  |  1.36  | 0.5617 
450  | 2.57  |  1.23  | 0.5888 
475  | 2.24  |  1.35  | 0.6186 
500  | 2.32  |  1.52  | 0.6449 
525  | 2.18  |  1.07  | 0.6730 
550  | 2.08  |  0.96  | 0.6965 
575  | 1.99  |  1.03  | 0.7226 
600  | 1.90  |  1.01  | 0.7472 
625  | 1.82  |  1.02  | 0.7737 
650  | 1.75  |  0.86  | 0.7974 
675  | 1.28  |  0.92  | 0.8234 
700  | 1.64  |  0.80  | 0.8490 
725

4-element Array{Any,1}:
 LastMain.Output(1000,53085,[0.0 0.0880732 … 1.13277 1.16704],[1.0 0.229355 … 0.011472 0.0112244],[1.0 0.108 … 0.0054965 0.00548314],"rademacher-10-10-sgn","max_iter") 
 LastMain.Output(1000,53085,[0.0 0.0158351 … 0.582131 0.59705],[1.0 0.234526 … 0.0131045 0.0127203],[1.0 0.11109 … 0.00662151 0.00636297],"rademacher-10-10","max_iter")
 LastMain.Output(1000,1222260,[0.0 0.00193524 … 0.274848 0.277471],[1.0 0.0124964 … 0.000123463 0.000118088],[1.0 0.0294147 … 0.000262751 0.000209458],"CG","max_iter") 
 LastMain.Output(1000,8820,[0.0 0.00185541 … 0.054431 0.0555454],[1.0 0.266163 … 0.00702874 0.00662517],[1.0 0.166018 … 0.00757432 0.00744022],"CD-10","max_iter")      

In [305]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename)-OUTPUTS.jld", "OUTPUTS", OUTPUTS)

In [306]:
#using GR
using Plots
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

rcv1_train.binary


In [ ]:
# Loading saved problem data
probname = "w8a"#"gisette_scale-ridge";  
test = load("../data/$(probname)-OUTPUTS.jld")
OUTPUTS = test["OUTPUTS"];